In [1]:
# pip --trusted-host=pypi.org --trusted-host=files.pythonhosted.org install pandas notebook plotly chart_studio requests beautifulsoup4
import bs4

In [2]:
import requests

In [3]:
requests.packages.urllib3.disable_warnings()

In [4]:
import pandas

In [5]:
import re

In [6]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.express as px

In [16]:
test={}

In [25]:
def get_year(year):
    try:
        doc = bs4.BeautifulSoup(requests.get(f"https://en.wikipedia.org/wiki/{year}_in_film", verify=False).content)
        for s in doc.select("sup"):
            s.extract()
            
        data = [t for t in [t for t in doc.find_all("table") if len(t.find_all("caption"))>0] if t.find("caption").get_text().strip().startswith(f"Highest-grossing films of {year}")]
        data = data[0]
        data2 = pandas.read_html(str(data))[0]
        data2["Year"] = year
        if "Domestic rentals" in data2.columns:
            field = "Domestic rentals"
        if "Box-office gross" in data2.columns:
            field = "Box-office gross"
        if "Worldwide gross" in data2.columns:
            field = "Worldwide gross"
        if "Worldwide gross (USD)" in data2.columns:
            field = "Worldwide gross (USD)"
        
        data2["Taking"] = data2[field]
        data2 = data2.drop(columns=[field])
        data2["test"] = data2["Taking"].str.extract('([0-9.,]+)')
        data2["Taking"] = data2["test"].str.replace(",","").astype("float")
        data2 = data2.drop(columns=["test"])
        return data2
    except Exception as a:
        print(a)
        print(data)
        return None

In [17]:
for y in range(1965,1986):
    test[y]=get_year(y)

In [19]:
for y in range(1986,2004):
    test[y]=get_year(y)

In [26]:
for y in range(2004,2010):
    test[y]=get_year(y)

In [27]:
data = pandas.concat([test[t] for t in test])
data["Distributor"] = data["Distributor"].fillna(data["Distributor(s)"])
data = data.drop(columns=["Distributor(s)"])

In [28]:
pandas.set_option('display.float_format', lambda x: '%.3f' % x)
df = data[["Year","Taking"]].groupby(["Year"]).agg(["mean","median","max"])
df = df.reset_index()
df.columns = ["year","mean","median","max"]

In [29]:
top = data[data.Rank==1].sort_values("Year")[["Title"]]
top = list(top.Title)
top

['The Sound of Music',
 'Hawaii',
 'The Graduate',
 'Funny Girl',
 'Butch Cassidy and the Sundance Kid',
 'Love Story',
 'Fiddler on the Roof',
 'The Godfather',
 'The Exorcist',
 'The Towering Inferno',
 'Jaws',
 'Rocky',
 'Star Wars',
 'Superman',
 'Kramer vs. Kramer',
 'The Empire Strikes Back',
 'Raiders of the Lost Ark',
 'E.T. the Extra-Terrestrial',
 'Return of the Jedi',
 'Beverly Hills Cop',
 'Back to the Future',
 'Top Gun',
 'Three Men and a Baby',
 'Rain Man',
 'Indiana Jones and the Last Crusade',
 'Ghost',
 'Terminator 2: Judgment Day',
 'Aladdin',
 'Jurassic Park',
 'The Lion King',
 'Die Hard with a Vengeance',
 'Independence Day',
 'Titanic',
 'Armageddon',
 'Star Wars: Episode I – The Phantom Menace',
 'Mission: Impossible 2',
 'Harry Potter and the Sorcerer’s Stone',
 'The Lord of the Rings: The Two Towers',
 'The Lord of the Rings: The Return of the King',
 'Shrek 2',
 'Harry Potter and the Goblet of Fire',
 "Pirates of the Caribbean: Dead Man's Chest",
 "Pirates of

In [30]:
fig = px.line(df,x="year", y=["mean","median","max"])
for t in range(len(top)):
    xx=list(df.year)[t]
    yy=list(df["max"])[t]
    tt=top[t]
    fig.add_annotation(x=xx, y=yy, text=tt, textangle=90)

fig.show()

In [18]:
test

{1965:    Rank                                           Title  \
 0     1                              The Sound of Music   
 1     2                                  Doctor Zhivago   
 2     3                                     Thunderball   
 3     4  Those Magnificent Men in their Flying Machines   
 4     5                                  The Great Race   
 5     6                                  That Darn Cat!   
 6     7                                      Cat Ballou   
 7     8                            What's New Pussycat?   
 8     9                                      Shenandoah   
 9    10                              Von Ryan's Express   
 
                                  Studio  Year    Taking  
 0                      20th Century Fox  1965  72000000  
 1                   Metro-Goldwyn-Mayer  1965  43000000  
 2        United Artists/Eon Productions  1965  27000000  
 3                      20th Century Fox  1965  14000000  
 4                 Warner Bros. Pictu